In [1]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import csv
import pandas as pd
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.84.231.154:8470


W0824 17:08:30.385006 140042744510336 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5161697567607345077),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 15013514628316835704),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14331371067235975315),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 17970072412034988948),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 13639913727209356923),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 15016154012244499900),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 10601350430831536356),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 4545879471716091328),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 2234474461560175557),
 _DeviceAttributes(/job:tpu_

In [2]:
import sys
from os import listdir
!rm -rf bert_finetune
!test -d bert_adapter || git clone https://github.com/junhahyung/adapter-bert bert_adapter
if not 'bert_adapter' in sys.path:
  sys.path += ['bert_adapter']
print(listdir("bert_adapter"))
# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_adapter'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 47 (delta 16), reused 36 (delta 8), pack-reused 0
Unpacking objects: 100% (47/47), done.


W0824 17:08:37.002433 140042744510336 deprecation_wrapper.py:119] From bert_adapter/optimization.py:90: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



['run_classifier.py', 'optimization_test.py', 'modeling_test.py', 'LICENSE', 'optimization.py', '__init__.py', 'run_classifier_with_tfhub.py', 'tokenization.py', 'README.md', 'modeling.py', 'requirements.txt', 'tokenization_test.py', '.git']


In [3]:
TASK = 'korean_sa'
assert TASK in ('MRPC', 'CoLA', 'korean_sa'), 'Undefined task'

BUCKET = 'mbertfinetune' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
TASK_DATA_DIR = 'gs://{}/data/{}'.format(BUCKET, TASK)
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
print(tf.gfile.ListDirectory(TASK_DATA_DIR))
OUTPUT_DIR = 'gs://{}/bert-adapter-tfhub-all_layers/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

BERT_MODEL = 'multi_cased_L-12_H-768_A-12'
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

CKPT_DIR = 'gs://{}/{}/bert_model.ckpt'.format(BUCKET, BERT_MODEL)
CONFIG_DIR = 'gs://{}/{}/bert_config.json'.format(BUCKET, BERT_MODEL)

***** Task data directory: gs://mbertfinetune/data/korean_sa *****
['korean_dev.csv', 'korean_train.csv']
***** Model output directory: gs://mbertfinetune/bert-adapter-tfhub-all_layers/models/korean_sa *****


In [4]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("한국어는 잘 안되는 것 같아요.")

W0824 17:08:55.822245 140042744510336 deprecation_wrapper.py:119] From bert_adapter/run_classifier_with_tfhub.py:151: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0824 17:08:56.167306 140042744510336 deprecation_wrapper.py:119] From bert_adapter/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



['한국', '##어는', '잘', '안', '##되는', '것', '같', '##아', '##요', '.']

In [5]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 3e-4 
NUM_TRAIN_EPOCHS = 3 
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500
# layer_wise_lr = (True, 0.3)
# layer_wise_lr = None 

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
  "korean_sa": run_classifier.KsaProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

processor.get_train_examples(TASK_DATA_DIR)
# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000 

def get_run_config(output_dir):  
  return tf.contrib.tpu.RunConfig(    
      cluster=tpu_cluster_resolver,    
      model_dir=output_dir,    
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,    
      tpu_config=tf.contrib.tpu.TPUConfig(        
          iterations_per_loop=ITERATIONS_PER_LOOP,        
          num_shards=NUM_TPU_CORES,        
          per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

W0824 17:08:56.588099 140042744510336 deprecation_wrapper.py:119] From bert_adapter/run_classifier.py:334: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [6]:
# Maybe don't need this because I'm not using TFHUB
# os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR
bert_config = modeling.BertConfig.from_json_file(CONFIG_DIR)

model_fn = run_classifier.model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),
    init_checkpoint=CKPT_DIR,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=True,
    use_one_hot_embeddings=True,
    use_all_layers=True
     )
# use_one_hot_embeddings is set as same value as use_tpu

estimator = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

W0824 17:09:00.173160 140042744510336 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f5dee47ab70>) includes params argument, but params are not passed to Estimator.


In [0]:
def model_train(estimator):
  print('Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))

In [8]:
model_train(estimator)

W0824 17:09:00.203823 140042744510336 deprecation_wrapper.py:119] From bert_adapter/run_classifier.py:916: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



Please wait...
***** Started training at 2019-08-24 17:09:01.629659 *****
  Num examples = 8961
  Batch size = 32


W0824 17:09:02.005680 140042744510336 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0824 17:09:05.837388 140042744510336 deprecation_wrapper.py:119] From bert_adapter/modeling.py:174: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0824 17:09:05.842920 140042744510336 deprecation_wrapper.py:119] From bert_adapter/modeling.py:487: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0824 17:09:05.878098 140042744510336 deprecation_wrapper.py:119] From bert_adapter/modeling.py:568: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.


***** Finished training at 2019-08-24 17:12:10.358515 *****


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))

In [10]:
model_eval(estimator)

***** Started evaluation at 2019-08-24 17:12:11.709460 *****
  Num examples = 2194
  Batch size = 8


W0824 17:12:20.720178 140042744510336 deprecation_wrapper.py:119] From bert_adapter/run_classifier.py:828: The name tf.metrics.accuracy is deprecated. Please use tf.compat.v1.metrics.accuracy instead.

W0824 17:12:20.741233 140042744510336 deprecation_wrapper.py:119] From bert_adapter/run_classifier.py:830: The name tf.metrics.mean is deprecated. Please use tf.compat.v1.metrics.mean instead.

W0824 17:12:22.759567 140042744510336 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


***** Finished evaluation at 2019-08-24 17:13:06.446557 *****
***** Eval results *****
  eval_accuracy = 0.58987224
  eval_loss = 1.0864862
  global_step = 840
  loss = 1.1014642
